# Importing phase

## Importing libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

## Importing the dataset

In [2]:
data_train = pd.read_csv("application_train.csv")

# Preparing the data

## Removing missing values

Removing rows with at least one missing value

In [3]:
data_train = data_train.dropna(axis = 0)

## Transforming to dummies

Transforming non-numerical variables to dummies

In [4]:
data_train = pd.get_dummies(data_train, columns = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'])

## Separating the target

We have to separate the 'TARGET' variable from the dataset, as it will be predicted

In [5]:
Y = data_train['TARGET']

Dropping the 'TARGET' variable from the original set

In [6]:
X = data_train.drop(columns = ['TARGET'])

## Splitting the data

We use scikit-learn to split the data into train and test sets

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# Building models

## Random Forest

Initiating the random forest model

In [8]:
randomforest = RandomForestClassifier(n_estimators = 1000)

Training the model

In [9]:
randomforest.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=1000)

Predicting targets for test set

In [10]:
targets = randomforest.predict(X_test)

Calculating accuracy using sklearn

In [11]:
print("Accuracy = ",metrics.accuracy_score(Y_test, targets))

Accuracy =  0.9339841933984193
